In [1]:
import camelot
import pandas as pd

# Updated Process

## 1. Create empty dataframe with proper column names

In [2]:
# This is where data will be added after cleaning. The final dataframe for export

In [3]:
df = pd.DataFrame(columns = [
    "Common_name",
    "Species",
    "BE", 
    "BB", 
    "DE", 
    "GS",
    "Vorzugshabitate",
    "Gefährdungsursachen"
])
df

,Common_name,Species,BE,BB,DE,GS,Vorzugshabitate,Gefährdungsursachen


In [4]:
df.shape

(0, 8)

## 2. Setup mechanism to store accuracy report : QA step

In [5]:
accuracy = []

## 3. Dictionary of dataframes on the go

In [6]:
# Dictionary to collect dataframes created from each table
d = {}

## 4. Adding the list of pages to be parsed

In [7]:
pages = []
i = 4
while i < 7:
    pages.append(i)
    i = i + 1
print(pages)

[4, 5, 6]


## 5. Parsing through the pages

In [8]:
for i in pages:
    # print page number
    print (i)
    # create dataframe with page number
    d[i] = pd.DataFrame()
    #read the page
    #tables = camelot.read_pdf('18_netzfl_print.pdf', 
    #                          pages= str(i), 
    #                          flavor='stream', edge_tol=1000, row_tol=10)
    tables = camelot.read_pdf('10_saeuge_print.pdf', pages= str(i), flavor='lattice')
    # add the accuracy report
    accuracy.append(tables[0].parsing_report)
    # assign the first table to a dataframe
    d[i] = tables[0].df
    print(d[i].shape)
# Print Done once all pages are parsed
print ("Done")

4
(12, 8)
5
(28, 8)
6
(28, 8)
Done


In [9]:
# Check accuracy reports for the process
accuracy

[{'accuracy': 98.92, 'whitespace': 29.17, 'order': 1, 'page': 4},
 {'accuracy': 98.65, 'whitespace': 18.75, 'order': 1, 'page': 5},
 {'accuracy': 99.56, 'whitespace': 23.66, 'order': 1, 'page': 6}]

# Data cleanup steps

In [10]:
# Check whether all dataframes are of the same size
for key in d:
    print(key)
    print(d[key].shape)

4
(12, 8)
5
(28, 8)
6
(28, 8)


In [11]:
# Removing empty dataframes
# del d[64]
# del d[74]

In [12]:
# Creates a sorted dictionary (sorted by key)
from collections import OrderedDict
d2 = OrderedDict(sorted(d.items()))

In [13]:
# Check whether the dictionary is sorted
for key in d2:
    print(key)

4
5
6


In [14]:
# Rename all the dataframes in the sorted dictionary
for key in d2:
    print(key)
    d2[key].columns = [  "Common_name",
    "Species",
    "BE", 
    "BB", 
    "DE", 
    "GS",
    "Vorzugshabitate",
    "Gefährdungsursachen"]
print("Rename completed for concat")

4
5
6
Rename completed for concat


In [15]:
# Concatanate all the non-empty dataframes in the sorted dictionary into the empty df
for key in d2:
    df = pd.concat([df, d2[key]], axis = 0)
df

,Common_name,Species,BE,BB,DE,GS,Vorzugshabitate,Gefährdungsursachen
0,Deutscher Name \nWissenschaftlicher Name \nBE\...,,,,,,,
1,Insektenfresser Insectivora,,,,,,,
2,Braunbrustigel,Erinaceus europaeus LINNAEUS,-,-,-,§,"W, P, O",
3,Feldspitzmaus*,Crocidura leucodon (HER-\nMANN)*,-,-,3,§,"RS, G, LA",
4,Gartenspitzmaus*,Crocidura suaveolens (PAL-\nLAS)*,-,-,3,§,"RS, OAD",
...,...,...,...,...,...,...,...,...
23,Damhirsch,Cervus dama LINNAEUS,-,-,-,,W,
24,Mufflon,Ovis ammon (PALLAS),-,-,-,,W,
25,Reh,Capreolus capreolus (LIN-\nNAEUS),-,-,-,,"W, G, L",
26,Rothirsch*,Cervus elaphus LINNAEUS*,0,-,,,W,"4d, 14b"


In [16]:
# Definition of extinction
df2 = df.loc[df["BE"] == "0"]
df2.reset_index(inplace = True)
df2 = df2.drop(['index'], axis=1)
df2.head()

,Common_name,Species,BE,BB,DE,GS,Vorzugshabitate,Gefährdungsursachen
0,Mopsfledermaus,Barbastella barbastellus \n(SCHREBER),0,1,1,"§§, II W, O",,"2c, 9, 14h"
1,Hamster,Cricetus cricetus (LINNAEUS),0,1,2,§§,L,"1a, 6"
2,Hausratte,Rattus rattus (LINNAEUS),0,2,-,,O,"2c, 4d"
3,Rothirsch*,Cervus elaphus LINNAEUS*,0,-,,,W,"4d, 14b"


In [ ]:
#df3 = df.loc[df["Bestand"] == "ex"]
#df3.reset_index(inplace = True)
#df3 = df3.drop(['index'], axis=1)
#df3.head()

In [17]:
df2.describe()

,Common_name,Species,BE,BB,DE,GS,Vorzugshabitate,Gefährdungsursachen
count,4,4,4,4,4,4,4,4
unique,4,4,1,3,4,3,4,4
top,Mopsfledermaus,Barbastella barbastellus \n(SCHREBER),0,1,1,,,"2c, 9, 14h"
freq,1,1,4,2,1,2,1,1


In [ ]:
#df3.describe()

In [18]:
# export the dataframe as CSV
df2.to_csv('Mammalia.csv', index=False)